In [2]:
# Imports
import os, sys
import django
from datetime import datetime
sys.path.append('../..') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from edc_appointment.models import Appointment
from flourish_caregiver.models import *
from flourish_child.models import *
from edc_registration.models import *
from edc_visit_schedule.models import *
from edc_constants.constants import *

In [3]:
# init data to be used

caregiver_subject_identifier = 'C142-040990642-6'
caregiver_screening_identifier = 'S99Y7REX'
child_subject_identifier = 'C142-040990642-6-10'
new_child_subject_identifier = 'B142-040990642-6-10'
new_subject_identifier = 'B142-040990642-6'
previous_subject_identifier = '056-1982862-0'



In [4]:
# estabilish what exists
caregiver_screening = ScreeningPriorBhpParticipants.objects.get(screening_identifier = caregiver_screening_identifier)
maternal_dataset = MaternalDataset.objects.get(study_maternal_identifier=previous_subject_identifier)
caregiver_consent = SubjectConsent.objects.get(subject_identifier=caregiver_subject_identifier)
child_consent = CaregiverChildConsent.objects.get(subject_identifier = child_subject_identifier)

In [6]:
# unassign screening to allow save the new screening

caregiver_screening.study_maternal_identifier = ''
caregiver_screening.save()

In [7]:
# unassign to allow the new caregiver pid to be saved and generate desire screening id

try:
    MaternalDataset.objects.get(subject_identifier = new_subject_identifier)
except MaternalDataset.DoesNotExist:
    maternal_dataset.subject_identifier = new_subject_identifier
    maternal_dataset.screening_identifier = ''
    maternal_dataset.save()

assert maternal_dataset.screening_identifier and maternal_dataset.screening_identifier != caregiver_screening.screening_identifier


In [47]:
# screening for the bio mother
# the clinic to be changed if necessary 

from copy import deepcopy

screening = None

try:

    screening = ScreeningPriorBhpParticipants.objects.get(subject_identifier = new_subject_identifier)
except ScreeningPriorBhpParticipants.DoesNotExist:

    screening = deepcopy(caregiver_screening)
    screening.id = None
    screening.slug = None
    screening.flourish_participation = 'interested'
    screening.screening_identifier = maternal_dataset.screening_identifier
    screening.subject_identifier = new_subject_identifier
    screening.is_consented = True
    screening.mother_alive = YES
    screening.save()
    

assert screening.mother_alive == YES
assert new_subject_identifier


In [48]:
from edc_constants.constants import FEMALE, YES, OTHER, OMANG
from edc_base.utils import get_utcnow
import datetime
import pytz


utc=pytz.UTC


new_mother_consent = None
biological_mother_subject_consent = deepcopy(caregiver_consent)

new_consent = None

try:
    
    new_consent = SubjectConsent.objects.get(subject_identifier = new_subject_identifier)
    
except SubjectConsent.DoesNotExist:

    
    new_consent = SubjectConsent()
    new_consent.screening_identifier = maternal_dataset.screening_identifier
    new_consent.subject_identifier = maternal_dataset.subject_identifier
    new_consent.first_name = maternal_dataset.first_name
    new_consent.last_name = maternal_dataset.last_name
    new_consent.initials = f'{maternal_dataset.first_name[0]}{ maternal_dataset.last_name[0]}'.upper()
    new_consent.language = 'tn'
    new_consent.recruit_source = 'BHP recruiter/clinician'
    new_consent.recruitment_clinic = 'Prior'
    new_consent.dob = datetime.date(1980, 1, 17)
    new_consent.is_dob_estimated = '-'
    new_consent.citizen = YES
    new_consent.gender = FEMALE
    new_consent.identity_type = 'country_id'
    new_consent.identity = '123456789'
    new_consent.confirm_identity = '123456789'
    new_consent.remain_in_study = YES
    new_consent.hiv_testing = NOT_APPLICABLE
    new_consent.breastfeed_intent = NOT_APPLICABLE
    new_consent.child_consent = YES
    new_consent.consent_reviewed = YES
    new_consent.study_questions = YES
    new_consent.assessment_score = YES
    new_consent.consent_signature = YES
    new_consent.consent_copy = YES
    new_consent.future_contact = YES
    new_consent.biological_caregiver = YES
    new_consent.consent_datetime = datetime.datetime(2022, 12, 14, 10, 47).replace(tzinfo=utc)
    new_consent.version = '2'
    new_consent.is_literate = YES
    new_consent.save()
    

if not new_consent.caregiverchildconsent_set.exists():
    new_consent.caregiverchildconsent_set.add(child_consent)
    new_consent.save()

assert new_consent
assert new_consent.subject_identifier == new_subject_identifier


In [49]:
try:
    registered_consent =  RegisteredSubject.objects.get(subject_identifier = new_subject_identifier)
except RegisteredSubject.DoesNotExist:
    pass
else:
    registered_consent.relative_identifier = child_subject_identifier
    registered_consent.save()

assert registered_consent.relative_identifier == child_subject_identifier


In [50]:
try:
    child_registered_consent =  RegisteredSubject.objects.get(subject_identifier = child_subject_identifier)
except RegisteredSubject.DoesNotExist:
    pass
else:
    child_registered_consent.relative_identifier = new_subject_identifier
    child_registered_consent.save()

assert child_registered_consent.relative_identifier == new_subject_identifier

In [51]:
maternal_dataset.subject_identifier = new_subject_identifier
maternal_dataset.save()

In [52]:
try:
    
    version = FlourishConsentVersion.objects.get(
        screening_identifier = maternal_dataset.screening_identifier,
    )
    
except FlourishConsentVersion.DoesNotExist:
    version = FlourishConsentVersion(
        screening_identifier = maternal_dataset.screening_identifier,
        version = '2',
        child_version = '2'
    )
    
    version.save()



assert version.screening_identifier == maternal_dataset.screening_identifier

In [53]:
from edc_appointment.models import Appointment as CaregiverAppointment
from flourish_child.models import Appointment as ChildAppointment

child_appointments = ChildAppointment.objects.filter(
    subject_identifier=child_subject_identifier,
    visit_code__gte=2001)

for child_appointment in child_appointments:
    
    assert int(child_appointment.visit_code) >= 2001
    
    try:
    
        caregiver_appointment = CaregiverAppointment.objects.get(
                subject_identifier = new_subject_identifier,
                visit_code__istartswith=child_appointment.visit_code)
        
    except CaregiverAppointment.DoesNotExist:
        pass
    else:
        assert caregiver_appointment.visit_code.startswith(child_appointment.visit_code)
        
        if caregiver_appointment.appt_datetime != child_appointment.appt_datetime:
            caregiver_appointment.appt_datetime = child_appointment.appt_datetime
            caregiver_appointment.save()
            
            
    

In [56]:
# Delete appointments not needed thus 2001M and 2002M

CaregiverAppointment.objects.filter(
    subject_identifier = new_subject_identifier,
    visit_code__in = ['2001M', '2002M']).delete()



(0, {})

In [58]:
caregiver_appointment.__dict__

{'_state': <django.db.models.base.ModelState at 0x7fb30364f700>,
 'created': datetime.datetime(2023, 2, 9, 13, 53, 12, 773701, tzinfo=<UTC>),
 'modified': datetime.datetime(2023, 2, 23, 8, 39, 5, 79685, tzinfo=<UTC>),
 'user_created': 'flourish',
 'user_modified': 'flourish',
 'hostname_created': 'flourish-dev',
 'hostname_modified': 'cyborg-2.local',
 'revision': '',
 'device_created': '99',
 'device_modified': '01',
 'id': UUID('5caf7a8b-888f-4c7e-a97b-c577d4a80652'),
 'site_id': 40,
 'subject_identifier': 'B142-040990642-6',
 'visit_schedule_name': 'b1_visit_schedule1',
 'schedule_name': 'b_sec_quart1_schedule1',
 'visit_code': '2009M',
 'visit_code_sequence': 0,
 'timepoint_status': 'open',
 'timepoint_opened_datetime': datetime.datetime(2024, 4, 4, 6, 7, 33, tzinfo=<UTC>),
 'timepoint_closed_datetime': None,
 'timepoint': Decimal('9.0'),
 'timepoint_datetime': datetime.datetime(2025, 4, 29, 13, 52, 58, tzinfo=<UTC>),
 'appt_close_datetime': None,
 'facility_name': '5-day clinic',
